# Code for chatbot using mongodb 

In [ ]:
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, MessagesState, START
from langgraph.checkpoint.mongodb import MongoDBSaver

# model = init_chat_model(model="anthropic:claude-3-5-haiku-latest")

from langchain_community.chat_models import ChatOllama
model = ChatOllama(model="llama3.1:8b")



DB_URI = "localhost:27017"
with MongoDBSaver.from_conn_string(DB_URI) as checkpointer:

    def call_model(state: MessagesState):
        response = model.invoke(state["messages"])
        return {"messages": response}

    builder = StateGraph(MessagesState)
    builder.add_node(call_model)
    builder.add_edge(START, "call_model")

    graph = builder.compile(checkpointer=checkpointer)

    config = {
        "configurable": {
            "thread_id": "2"
        }
    }

    for chunk in graph.stream(
        {"messages": [{"role": "user", "content": "hi! I'm uday"}]},
        config,
        stream_mode="values"
    ):
        chunk["messages"][-1].pretty_print()

    for chunk in graph.stream(
        {"messages": [{"role": "user", "content": "what's my name?"}]},
        config,
        stream_mode="values"
    ):
        chunk["messages"][-1].pretty_print()

================================ Human Message =================================

what's my name?
================================== Ai Message ==================================

Your name is... UDAY! (I remember, we established that at the beginning of our conversation)


# JD Generator

In [6]:
import os
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain_openai import AzureChatOpenAI

# Load environment variables
load_dotenv()

# Initialize Azure OpenAI GPT-4
model = AzureChatOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    temperature=0.3
)

template = """
You are an expert HR copywriter with experience in the tech industry. Your task is to write a clear, engaging, and comprehensive job description based on the provided keywords.

**Job Title:** {job_title}

**Company Tone:** {company_tone}

**Key Responsibilities:**
{responsibilities}

**Core Skills Required:**
{skills}

**Years of Experience:** {experience} years

Based on the information above, please generate a full job description. The description must include the following sections in Markdown format:
- ## About the Role
- ## Key Responsibilities
- ## Required Qualifications
- ## Preferred Qualifications (You can infer these based on the role)
- ## Why You'll Love Working With Us

Ensure the language and tone match the specified company tone. Do not invent benefits or company details unless they are general and positive (e.g., "collaborative environment").
"""
prompt = PromptTemplate(
    input_variables=["job_title", "company_tone", "responsibilities", "skills", "experience"],
    template=template
)

ModuleNotFoundError: No module named 'langchain.prompts'

In [ ]:
# Using modern LangChain syntax (no deprecation warnings)
chain = prompt | model


C:\Users\udayt\AppData\Local\Temp\ipykernel_18972\2045290583.py:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=model,prompt=prompt)


In [ ]:
user_input = {
    "job_title": "Senior Backend Engineer (Python)",
    "company_tone": "Professional yet approachable",
    "responsibilities": "Design and build scalable APIs, database management, mentor junior engineers, lead code reviews, collaborate with frontend teams",
    "skills": "Python, FastAPI, SQL (PostgreSQL), Docker, Kubernetes, AWS, CI/CD",
    "experience": "5-7"
}

print("🔄 Generating Job Description with Azure OpenAI GPT-4...")
response = chain.invoke(user_input)

In [ ]:
# Extract the content from the Azure OpenAI response
text = response.content

In [25]:
print(text)

## Senior Backend Engineer (Python) Job Description
### About the Role

We're seeking an experienced Senior Backend Engineer to join our team, where you'll play a key role in designing and building scalable APIs that power our applications. As a seasoned engineer with 5-7 years of experience, you'll be responsible for leading code reviews, mentoring junior engineers, and collaborating closely with frontend teams to deliver high-quality software solutions.

### Key Responsibilities

* Design and build scalable APIs using Python and FastAPI
* Manage databases efficiently using PostgreSQL and SQL
* Mentor junior engineers in best practices and coding standards
* Lead code reviews to ensure high-quality code and adherence to company guidelines
* Collaborate closely with frontend teams to deliver seamless user experiences
* Ensure smooth deployment and scaling of applications using Docker, Kubernetes, and AWS

### Required Qualifications

* 5-7 years of experience as a Backend Engineer, pre

In [31]:
from fpdf import FPDF

def save_text_to_pdf(markdown_text, filename="Job.pdf"):
    """
    Parses markdown-like text and saves it as a PDF file.
    Handles titles (##), bullet points (-), and paragraphs.
    """
    pdf = FPDF()
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)
    
    # Use a common font that supports a range of characters
    pdf.set_font("Arial", size=12)

    lines = markdown_text.split('\n')

    for line in lines:
        line = line.strip()
        if line.startswith('### '):
            # H2 Title
            pdf.set_font("Arial", 'B', 16)
            # Remove the '## ' prefix before writing
            pdf.multi_cell(0, 10, line[3:])
            pdf.ln(4) # Add a little space after the title
        elif line.startswith('- '):
            # Bullet point
            pdf.set_font("Arial", '', 12)
            # Use multi_cell for automatic line wrapping of long bullet points
            pdf.multi_cell(0, 6, f"  • {line[2:]}")
            pdf.ln(1) # Small space between bullet points
        elif line: # Check if line is not empty
            # Regular paragraph
            pdf.set_font("Arial", '', 12)
            pdf.multi_cell(0, 6, line)
            pdf.ln(3) # Space after a paragraph

    try:
        pdf.output(filename)
        print(f"\n✅ Successfully saved Job Description as '{filename}'")
    except Exception as e:
        print(f"❌ Error saving PDF: {e}")

In [37]:

def save_text_to_pdf(markdown_text, filename="job_description.pdf"):
    """
    Parses markdown-like text and saves it as a PDF file using fpdf2.
    Handles a main title (##), subtitles (###), bullet points (*), and paragraphs.
    """
    pdf = FPDF()
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)
    
    # Use a standard, universally available font like Arial.
    pdf.set_font("Arial", size=12)

    lines = markdown_text.split('\n')

    for line in lines:
        line = line.strip()
        if line.startswith('## '):
            # H2 Main Title
            pdf.set_font("Arial", 'B', 20)
            pdf.multi_cell(0, 12, line[3:], align='C') # Centered title
            pdf.ln(10) 
        elif line.startswith('### '):
            # H3 Subtitle
            pdf.set_font("Arial", 'B', 16)
            pdf.multi_cell(0, 10, line[4:])
            pdf.ln(4)
        elif line.startswith('* '):
            # Bullet point
            pdf.set_font("Arial", '', 12)
            # Use a simple ASCII character for the bullet to avoid encoding errors.
            pdf.multi_cell(0, 6, f"  - {line[2:]}")
            pdf.ln(1) # Small space between bullet points
        elif line: # Check if line is not empty
            # Regular paragraph
            pdf.set_font("Arial", '', 12)
            pdf.multi_cell(0, 6, line)
            pdf.ln(3) # Space after a paragraph

    try:
        pdf.output(filename)
        print(f"\n✅ Successfully saved Job Description as '{filename}'")
    except Exception as e:
        print(f"❌ Error saving PDF: {e}")


In [38]:
save_text_to_pdf(text)


✅ Successfully saved Job Description as 'job_description.pdf'


# Resume screener & candidate matcher

In [4]:
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List, Dict, Literal

# --- 1. Pydantic Models for Structured Data ---
# These models define the exact structure of the data we want to extract and generate.

class JobDescription(BaseModel):
    """Structured data extracted from a job description."""
    required_skills: List[str] = Field(description="A list of essential skills required for the job.")
    preferred_skills: List[str] = Field(description="A list of preferred but not mandatory skills.")
    years_of_experience: int = Field(description="The minimum number of years of experience required.")

class CandidateResume(BaseModel):
    """Structured data extracted from a candidate's resume."""
    name: str = Field(description="The full name of the candidate.")
    skills: List[str] = Field(description="A list of skills possessed by the candidate.")
    work_experience: List[Dict] = Field(description="A list of previous work experiences, including job title, company, and duration in years.")
    total_experience_years: int = Field(description="The total number of years of professional experience.")

class ScreeningResult(BaseModel):
    """The final output of the screening process."""
    match_score: int = Field(description="A score from 0 to 100 representing the candidate's suitability.", ge=0, le=100)
    summary: str = Field(description="A concise summary explaining the reasoning behind the score, highlighting strengths and weaknesses.")
    status: Literal["Strong Match", "Potential Fit", "Not a Fit"] = Field(description="The final recommendation status.")


# --- 2. The AI Core Logic ---

class ResumeScreenerAI:
    def __init__(self, model_name="llama3.1:8b"):
        """Initializes the AI Screener with the specified Ollama model."""
        try:
            self.llm = Ollama(model=model_name, temperature=0.0, format="json")
            print(f"✅ LLM model '{model_name}' initialized successfully.")
        except Exception as e:
            print(f"❌ Error initializing Ollama model: {e}")
            raise

    def _extract_structured_data(self, text: str, pydantic_model):
        """A generic function to extract structured data from text using a Pydantic model."""
        parser = PydanticOutputParser(pydantic_object=pydantic_model)
        
        prompt = PromptTemplate(
            template="You are an expert HR data analyst. Extract the relevant information from the document below and format it according to the provided JSON schema.\n{format_instructions}\nDocument:\n{document_text}",
            input_variables=["document_text"],
            partial_variables={"format_instructions": parser.get_format_instructions()},
        )
        
        chain = LLMChain(llm=self.llm, prompt=prompt)
        
        try:
            output = chain.invoke({"document_text": text})
            parsed_output = parser.parse(output['text'])
            return parsed_output
        except Exception as e:
            print(f"❌ Error during data extraction: {e}")
            return None

    def screen_resume(self, jd_text: str, resume_text: str):
        """
        Performs the two-step screening process: extraction and comparison.
        """
        print("\n--- Step 1: Extracting structured data from documents ---")
        
        # Extract data from Job Description
        jd_data = self._extract_structured_data(jd_text, JobDescription)
        if not jd_data:
            print("❌ Failed to process Job Description.")
            return None
        print("✅ Job Description data extracted:")
        print(jd_data.model_dump_json(indent=2))

        # Extract data from Resume
        resume_data = self._extract_structured_data(resume_text, CandidateResume)
        if not resume_data:
            print("❌ Failed to process Resume.")
            return None
        print("\n✅ Resume data extracted:")
        print(resume_data.model_dump_json(indent=2))

        print("\n--- Step 2: Comparing data and generating screening result ---")
        
        comparison_parser = PydanticOutputParser(pydantic_object=ScreeningResult)

        comparison_prompt = PromptTemplate(
            template="""
            You are an expert Senior Technical Recruiter with 15 years of experience.
            Analyze the candidate's resume against the job description based on the structured data provided.
            Provide a match score, a concise summary of your analysis, and a final status.
            
            Job Requirements:
            {jd_json}
            
            Candidate's Resume:
            {resume_json}
            
            {format_instructions}
            """,
            input_variables=["jd_json", "resume_json"],
            partial_variables={"format_instructions": comparison_parser.get_format_instructions()},
        )
        
        comparison_chain = LLMChain(llm=self.llm, prompt=comparison_prompt)
        
        try:
            result_output = comparison_chain.invoke({
                "jd_json": jd_data.model_dump_json(),
                "resume_json": resume_data.model_dump_json()
            })
            final_result = comparison_parser.parse(result_output['text'])
            return final_result
        except Exception as e:
            print(f"❌ Error during comparison: {e}")
            return None


In [5]:

SAMPLE_JOB_DESCRIPTION = """
Job Title: Senior Python Developer

We are looking for a Senior Python Developer with at least 5 years of experience.
The ideal candidate must be proficient in Python, FastAPI, and PostgreSQL.
Experience with Docker and AWS is highly preferred. Key responsibilities include
developing backend services and mentoring junior engineers.
"""

SAMPLE_RESUME = """
Name: John Doe
Email: john.doe@email.com

Summary:
A skilled software engineer with 6 years of experience in backend development.

Skills:
- Python, Django, Flask, FastAPI
- PostgreSQL, MySQL, Redis
- Docker, Kubernetes, Jenkins
- Git, JIRA

Work Experience:
- Senior Software Engineer at Tech Solutions Inc. (3 years)
    - Led a team to build a microservices architecture using FastAPI and Docker.
- Software Engineer at Data Corp. (3 years)
    - Developed and maintained APIs using Python and Django.
"""

print("--- Initializing AI Resume Screener ---")

try:
    screener = ResumeScreenerAI()
    
    final_screening = screener.screen_resume(
        jd_text=SAMPLE_JOB_DESCRIPTION,
        resume_text=SAMPLE_RESUME
    )
    
    if final_screening:
        print("\n\n--- 🚀 Final Screening Result ---")
        print("---------------------------------")
        print(final_screening.model_dump_json(indent=2))
        
except Exception as e:
    print(f"\nAn unexpected error occurred during the main execution: {e}")
    print("Please ensure Ollama is running and accessible.")


--- Initializing AI Resume Screener ---
✅ LLM model 'llama3.1:8b' initialized successfully.

--- Step 1: Extracting structured data from documents ---
✅ Job Description data extracted:
{
  "required_skills": [
    "Python",
    "FastAPI",
    "PostgreSQL"
  ],
  "preferred_skills": [
    "Docker",
    "AWS"
  ],
  "years_of_experience": 5
}

✅ Resume data extracted:
{
  "name": "John Doe",
  "skills": [
    "Python, Django, Flask, FastAPI",
    "PostgreSQL, MySQL, Redis",
    "Docker, Kubernetes, Jenkins",
    "Git, JIRA"
  ],
  "work_experience": [
    {
      "job_title": "Senior Software Engineer",
      "company": "Tech Solutions Inc.",
      "duration_years": 3
    },
    {
      "job_title": "Software Engineer",
      "company": "Data Corp.",
      "duration_years": 3
    }
  ],
  "total_experience_years": 6
}

--- Step 2: Comparing data and generating screening result ---


--- 🚀 Final Screening Result ---
---------------------------------
{
  "match_score": 80,
  "summary": "Th

# AI-Powered Initiat Screening Interviewer

In [34]:
import os
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List, Dict, TypedDict, Optional, Literal

from langgraph.graph import StateGraph, END

# Load environment variables from .env file
load_dotenv()

# --- 1. Define the State for our Graph ---
# This TypedDict will be the "memory" of our interview agent.

class InterviewState(TypedDict):
    job_description: str
    interview_plan: Optional[List[str]]
    conversation_history: List[Dict[str, str]]
    evaluations: List[Dict]
    current_question: Optional[str]
    final_summary: Optional[str]
    max_questions: int

# --- 2. Define Pydantic Models for Structured Output ---

class InterviewPlan(BaseModel):
    topics: List[str] = Field(description="A list of 3-5 key technical and behavioral topics to cover in an interview.")

class Question(BaseModel):
    question: str = Field(description="The next question to ask the candidate.")

class Evaluation(BaseModel):
    rating: int = Field(description="A rating of the candidate's answer from 1 (poor) to 5 (excellent).")
    feedback: str = Field(description="A brief justification for the rating, explaining what was good or could be improved.")

class Summary(BaseModel):
    recommendation: Literal["Proceed", "Hold", "Reject"] = Field(description="The final hiring recommendation.")
    summary_text: str = Field(description="A comprehensive summary of the candidate's performance during the interview.")


# --- 3. The AI Core Logic (Nodes of the Graph) ---

class InterviewerAI:
    def __init__(self):
        # Initialize the Azure Chat OpenAI model
        try:
            self.llm = AzureChatOpenAI(
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
                api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
                azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                temperature=0.1,
                model_kwargs={
                    "response_format": {"type": "json_object"},
                }
            )
            print("✅ Azure OpenAI model initialized successfully.")
        except Exception as e:
            print(f"❌ Failed to initialize Azure OpenAI model. Please check your .env file and credentials.")
            print(f"Error: {e}")
            raise

    def generate_interview_plan(self, state: InterviewState):
        print("--- Node: Generating Interview Plan ---")
        jd = state['job_description']
        parser = PydanticOutputParser(pydantic_object=InterviewPlan)

        template = """
        You are a senior hiring manager. Read the job description and generate a list of 5 key topics to discuss.

        You MUST format your output as a JSON object with a single key "topics".

        Example Format:
        {{
          "topics": ["Python fundamentals", "FastAPI experience", "Database knowledge", "Team collaboration", "Problem-solving skills"]
        }}

        Job Description:
        {jd}
        """
        prompt = PromptTemplate(template=template, input_variables=["jd"])
        chain = prompt | self.llm | parser
        plan = chain.invoke({"jd": jd})
        return {"interview_plan": plan.topics}

    def generate_question(self, state: InterviewState):
        print("--- Node: Generating Question ---")
        history = state['conversation_history']
        plan = state['interview_plan']
        evals = state['evaluations']
        
        instruction = "You are an expert interviewer. "
        if evals and evals[-1]['rating'] < 3:
             instruction += f"The candidate's previous answer was weak. Ask a follow-up question to probe deeper into the topic of: '{history[-1]['question']}'."
        else:
            next_topic = plan[len(evals)] if len(evals) < len(plan) else "a final concluding question"
            instruction += f"Based on the interview plan, ask the next question about the topic: '{next_topic}'."
            
        parser = PydanticOutputParser(pydantic_object=Question)
        template = """
        {instruction}

        You MUST format your output as a JSON object with a single key "question".

        Example Format:
        {{
            "question": "Can you describe a challenging project you worked on using FastAPI?"
        }}

        Conversation History:
        {history}
        """
        prompt = PromptTemplate(template=template, input_variables=["instruction", "history"])
        chain = prompt | self.llm | parser
        question_obj = chain.invoke({"instruction": instruction, "history": history})
        return {"current_question": question_obj.question}

    def evaluate_answer(self, state: InterviewState):
        print("--- Node: Evaluating Answer ---")
        question = state['current_question']
        
        candidate_answer = input(f"🤖 AI: {question}\n👤 You: ")

        history = state['conversation_history']
        history.append({"question": question, "answer": candidate_answer})
        
        parser = PydanticOutputParser(pydantic_object=Evaluation)
        template = """
        You are an expert interview evaluator. Evaluate the candidate's answer based on the question asked.

        You MUST format your output as a JSON object with the keys "rating" and "feedback".

        Example Format:
        {{
            "rating": 4,
            "feedback": "The candidate provided a solid, real-world example and clearly explained the technical challenges."
        }}

        Question:
        {question}
        
        Candidate's Answer:
        {answer}
        """
        prompt = PromptTemplate(template=template, input_variables=["question", "answer"])
        chain = prompt | self.llm | parser
        evaluation = chain.invoke({"question": question, "answer": candidate_answer})

        evals = state['evaluations']
        evals.append(evaluation.model_dump())
        
        return {"conversation_history": history, "evaluations": evals}

    def summarize_interview(self, state: InterviewState):
        print("--- Node: Summarizing Interview ---")
        history = state['conversation_history']
        evals = state['evaluations']
        
        parser = PydanticOutputParser(pydantic_object=Summary)
        template = """
        You are a senior hiring manager. Based on the entire interview, write a final summary and provide a hiring recommendation.

        You MUST format your output as a JSON object with the keys "recommendation" and "summary_text".
        The "recommendation" value MUST be one of the following exact strings: "Proceed", "Hold", or "Reject".

        Example Format:
        {{
            "recommendation": "Proceed",
            "summary_text": "The candidate demonstrated strong technical skills in Python and FastAPI. They communicated effectively and showed good problem-solving abilities. Recommend proceeding to the next round."
        }}

        Interview Transcript:
        {history}

        Evaluations:
        {evals}
        """
        prompt = PromptTemplate(template=template, input_variables=["history", "evals"])
        chain = prompt | self.llm | parser
        summary = chain.invoke({"history": history, "evals": evals})
        return {"final_summary": summary.model_dump_json(indent=2)}

# --- 4. The Routing Logic (Edges) ---

def route_after_evaluation(state: InterviewState):
    print("--- Edge: Routing after evaluation ---")
    evals = state['evaluations']
    if len(evals) >= state['max_questions']:
        print("Decision: End of interview. Moving to summarize.")
        return "summarize"
    else:
        print("Decision: Continue interview. Moving to generate next question.")
        return "generate_question"

# --- 5. Build and Run the Graph ---



In [35]:
if __name__ == "__main__":
    SAMPLE_JOB_DESCRIPTION = """
    Job Title: Senior Python Developer
    We are looking for a Senior Python Developer with at least 5 years of experience.
    The ideal candidate must be proficient in Python, FastAPI, and PostgreSQL.
    Experience with Docker and AWS is highly preferred. Key responsibilities include
    developing backend services and mentoring junior engineers. The role requires strong
    problem-solving skills and excellent team collaboration.
    """
    
    ai_interviewer = InterviewerAI()

    # Define the graph
    workflow = StateGraph(InterviewState)

    # Add the nodes
    workflow.add_node("generate_plan", ai_interviewer.generate_interview_plan)
    workflow.add_node("generate_question", ai_interviewer.generate_question)
    workflow.add_node("evaluate_answer", ai_interviewer.evaluate_answer)
    workflow.add_node("summarize", ai_interviewer.summarize_interview)

    # Set the entry point
    workflow.set_entry_point("generate_plan")

    # Add the edges
    workflow.add_edge("generate_plan", "generate_question")
    workflow.add_edge("generate_question", "evaluate_answer")
    workflow.add_conditional_edges(
        "evaluate_answer",
        route_after_evaluation,
        {
            "summarize": "summarize",
            "generate_question": "generate_question"
        }
    )
    workflow.add_edge("summarize", END)

    # Compile the graph
    app = workflow.compile()

    # Run the interview
    print("\n--- Starting AI Interview ---")
    initial_state = {
        "job_description": SAMPLE_JOB_DESCRIPTION,
        "conversation_history": [],
        "evaluations": [],
        "max_questions": 4 # Let's do a 4-question interview
    }

    # Use stream to see the output of each step
    final_state_value = None
    for output in app.stream(initial_state):
        # The key is the name of the node that just ran
        for key, value in output.items():
            print(f"\nOutput from node '{key}':")
            print("---")
            final_state_value = value # Continuously update to get the last valid state

    # Access the final summary from the last known state
    if final_state_value:
        final_summary = final_state_value.get('final_summary')
        print("\n\n--- 🚀 Final Interview Summary ---")
        print("---------------------------------")
        print(final_summary)
    else:
        print("\n--- Interview could not be completed. ---")

✅ Azure OpenAI model initialized successfully.

--- Starting AI Interview ---
--- Node: Generating Interview Plan ---

Output from node 'generate_plan':
---
--- Node: Generating Question ---

Output from node 'generate_question':
---
--- Node: Evaluating Answer ---
--- Edge: Routing after evaluation ---
Decision: Continue interview. Moving to generate next question.

Output from node 'evaluate_answer':
---
--- Node: Generating Question ---

Output from node 'generate_question':
---
--- Node: Evaluating Answer ---
--- Edge: Routing after evaluation ---
Decision: Continue interview. Moving to generate next question.

Output from node 'evaluate_answer':
---
--- Node: Generating Question ---

Output from node 'generate_question':
---
--- Node: Evaluating Answer ---
--- Edge: Routing after evaluation ---
Decision: Continue interview. Moving to generate next question.

Output from node 'evaluate_answer':
---
--- Node: Generating Question ---

Output from node 'generate_question':
---
--- Nod

In [3]:
import os
from dotenv import load_dotenv

# A single script to test both Whisper and Speech Service credentials.

def test_whisper_config():
    """Tests the Azure Whisper (Speech-to-Text) configuration."""
    print("\n--- 🎤 Testing Whisper (Speech-to-Text) ---")
    
    # 1. Check for necessary environment variables
    required_vars = [
        "AZURE_OPENAI_API_KEY",
        "AZURE_OPENAI_ENDPOINT",
        "AZURE_OPENAI_API_VERSION",
        "AZURE_OPENAI_WHISPER_DEPLOYMENT_NAME"
    ]
    if not all(os.getenv(var) for var in required_vars):
        print("❌ FAILED: Missing one or more Azure OpenAI environment variables for Whisper.")
        return False
    
    print("✔️ All Whisper variables found.")
    
    # 2. Attempt to make an API call
    try:
        from openai import AzureOpenAI
        import io, wave

        client = AzureOpenAI(
            api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        )
        
        # Create a dummy silent WAV file in memory to send for transcription
        audio_buffer = io.BytesIO()
        with wave.open(audio_buffer, 'wb') as wf:
            wf.setnchannels(1); wf.setsampwidth(2); wf.setframerate(16000)
            wf.writeframes(b'\x00' * 1600) # 0.1 seconds of silence
        audio_buffer.seek(0)
        audio_buffer.name = "test.wav"
        
        print("... Contacting Whisper deployment...")
        client.audio.transcriptions.create(
            model=os.getenv("AZURE_OPENAI_WHISPER_DEPLOYMENT_NAME"),
            file=audio_buffer
        )
        print("✅ SUCCESS: Whisper configuration is correct.")
        return True

    except Exception as e:
        print(f"❌ FAILED: Could not connect to Whisper service. Error: {e}")
        return False


def test_speech_services_config():
    """Tests the Azure Speech Services (Text-to-Speech) configuration."""
    print("\n--- 🗣️ Testing Speech Services (Text-to-Speech) ---")

    # 1. Check for necessary environment variables
    required_vars = ["AZURE_SPEECH_KEY", "AZURE_SPEECH_REGION"]
    if not all(os.getenv(var) for var in required_vars):
        print("❌ FAILED: Missing AZURE_SPEECH_KEY or AZURE_SPEECH_REGION.")
        return False

    print("✔️ All Speech Service variables found.")

    # 2. Attempt to make an API call
    try:
        import azure.cognitiveservices.speech as speechsdk
        
        speech_key = os.getenv("AZURE_SPEECH_KEY")
        speech_region = os.getenv("AZURE_SPEECH_REGION")
        
        speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=speech_region)
        
        # We don't need to actually play the audio, just confirm the API call works.
        # So, we use a null audio output configuration.
        audio_config = speechsdk.audio.AudioOutputConfig(use_default_speaker=False)

        synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=audio_config)

        print("... Contacting Speech service...")
        result = synthesizer.speak_text_async("This is a test.").get()
        
        if result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
            print("✅ SUCCESS: Speech Services configuration is correct.")
            return True
        else:
            cancellation_details = result.cancellation_details
            print(f"❌ FAILED: Speech synthesis canceled: {cancellation_details.reason}")
            if cancellation_details.reason == speechsdk.CancellationReason.Error:
                print(f"   Error details: {cancellation_details.error_details}")
            return False

    except Exception as e:
        print(f"❌ FAILED: Could not connect to Speech service. Error: {e}")
        return False


if __name__ == "__main__":
    print("--- 🧪 Starting Full Azure Voice Environment Test ---")
    load_dotenv()
    
    whisper_ok = test_whisper_config()
    speech_ok = test_speech_services_config()
    
    print("\n--- 📊 Test Summary ---")
    if whisper_ok and speech_ok:
        print("🎉 All configurations are correct! Your application should work.")
    else:
        print("🚫 One or more tests failed. Please review the errors above and check your .env file.")

--- 🧪 Starting Full Azure Voice Environment Test ---

--- 🎤 Testing Whisper (Speech-to-Text) ---
✔️ All Whisper variables found.
... Contacting Whisper deployment...
❌ FAILED: Could not connect to Whisper service. Error: Error code: 404 - {'error': {'code': '404', 'message': 'Resource not found'}}

--- 🗣️ Testing Speech Services (Text-to-Speech) ---
✔️ All Speech Service variables found.
❌ FAILED: Could not connect to Speech service. Error: default speaker needs to be explicitly activated

--- 📊 Test Summary ---
🚫 One or more tests failed. Please review the errors above and check your .env file.


In [ ]:
import os
from openai import AzureOpenAI

endpoint = "https://mazenetai23-7805-resource.cognitiveservices.azure.com/"
model_name = "gpt-4o-mini"
deployment = "gpt-4o-mini"

subscription_key = "<your-api-key>"
api_version = "2024-12-01-preview"

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=subscription_key,
)

response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": "I am going to Paris, what should I see?",
        }
    ],
    max_tokens=4096,
    temperature=1.0,
    top_p=1.0,
    model=deployment
)

print(response.choices[0].message.content)

In [1]:
"""
TalentFlow AI - Complete Azure OpenAI Configuration Test
Test all API keys and endpoints used in the application
"""

import os
from dotenv import load_dotenv
from openai import AzureOpenAI
import time

# Load environment variables
load_dotenv()

# Configuration check
def check_env_vars():
    """Check if all required environment variables are set"""
    print("="*70)
    print("  🔍 Checking Environment Variables")
    print("="*70)
    
    required_vars = {
        "AZURE_OPENAI_API_KEY": "API Key for Azure OpenAI",
        "AZURE_OPENAI_ENDPOINT": "Endpoint URL",
        "AZURE_OPENAI_API_VERSION": "API Version",
        "AZURE_OPENAI_CHAT_DEPLOYMENT_NAME": "Chat Model Deployment (GPT-4)"
    }
    
    missing = []
    for var, desc in required_vars.items():
        value = os.getenv(var)
        if not value or value.startswith("your_"):
            print(f"  ❌ {var}: NOT CONFIGURED")
            print(f"     Description: {desc}")
            missing.append(var)
        else:
            # Mask sensitive data
            if "KEY" in var:
                masked = value[:8] + "..." + value[-4:] if len(value) > 12 else "***"
                print(f"  ✅ {var}: {masked}")
            else:
                print(f"  ✅ {var}: {value}")
    
    print()
    
    if missing:
        print(f"⚠️  Missing variables: {', '.join(missing)}")
        return False
    else:
        print("✅ All environment variables are configured!")
    
    return True

# Initialize Azure OpenAI client
def init_client():
    """Initialize Azure OpenAI client"""
    try:
        client = AzureOpenAI(
            api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
        )
        return client
    except Exception as e:
        print(f"❌ Failed to initialize client: {e}")
        return None

# Test 1: Basic Connection Test
def test_connection(client):
    """Test basic connection to Azure OpenAI"""
    print("\n" + "="*70)
    print("  🌐 Test 1: Basic Connection")
    print("="*70)
    
    try:
        deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
        
        print(f"Deployment: {deployment}")
        print("Testing connection with a simple prompt...")
        
        response = client.chat.completions.create(
            model=deployment,
            messages=[
                {"role": "user", "content": "Say 'Hello TalentFlow AI!' if you can hear me."}
            ],
            max_tokens=50,
            temperature=0
        )
        
        result = response.choices[0].message.content
        print(f"\n✅ Connection successful!")
        print(f"Response: {result}")
        return True
        
    except Exception as e:
        print(f"\n❌ Connection failed!")
        print(f"Error: {str(e)}")
        return False

# Test 2: JD Generator Endpoint Test
def test_jd_generation(client):
    """Test JD generation functionality"""
    print("\n" + "="*70)
    print("  📝 Test 2: Job Description Generation")
    print("="*70)
    
    try:
        deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
        
        template = """You are an expert HR copywriter. Generate a brief job description for:
Job Title: Software Engineer
Required Skills: Python, FastAPI
Experience: 3 years

Provide a 2-sentence job description."""

        print("Testing JD generation with sample data...")
        
        response = client.chat.completions.create(
            model=deployment,
            messages=[{"role": "user", "content": template}],
            max_tokens=200,
            temperature=0.3
        )
        
        result = response.choices[0].message.content
        print(f"\n✅ JD Generation successful!")
        print(f"Generated JD preview:\n{result[:150]}...")
        return True
        
    except Exception as e:
        print(f"\n❌ JD Generation failed!")
        print(f"Error: {str(e)}")
        return False

# Test 3: AI Interview Endpoint Test (JSON Mode)
def test_interview_json_mode(client):
    """Test AI interview with JSON output"""
    print("\n" + "="*70)
    print("  🎤 Test 3: AI Interview (JSON Mode)")
    print("="*70)
    
    try:
        deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
        
        prompt = """You are an AI interviewer. Generate a technical interview question.
You MUST format your output as a JSON object with a single key "question".

Example Format:
{
    "question": "Can you explain the difference between async and sync programming?"
}"""

        print("Testing interview question generation with JSON mode...")
        
        response = client.chat.completions.create(
            model=deployment,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=200,
            temperature=0.1,
            response_format={"type": "json_object"}
        )
        
        result = response.choices[0].message.content
        print(f"\n✅ AI Interview (JSON mode) successful!")
        print(f"JSON Response:\n{result}")
        
        # Verify it's valid JSON
        import json
        parsed = json.loads(result)
        print(f"✅ Valid JSON - Question extracted: {parsed.get('question', 'N/A')[:80]}...")
        return True
        
    except Exception as e:
        print(f"\n❌ AI Interview (JSON mode) failed!")
        print(f"Error: {str(e)}")
        return False

# Test 4: Rate Limit and Quota Check
def test_rate_limits(client):
    """Test rate limits with multiple requests"""
    print("\n" + "="*70)
    print("  ⚡ Test 4: Rate Limits & Performance")
    print("="*70)
    
    try:
        deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
        
        print("Sending 3 rapid requests to test rate limits...")
        
        for i in range(3):
            start = time.time()
            response = client.chat.completions.create(
                model=deployment,
                messages=[{"role": "user", "content": f"Count to {i+1}"}],
                max_tokens=20
            )
            elapsed = time.time() - start
            print(f"  Request {i+1}: ✅ Success ({elapsed:.2f}s)")
            time.sleep(0.5)  # Small delay between requests
        
        print("\n✅ Rate limit test passed! No throttling detected.")
        return True
        
    except Exception as e:
        print(f"\n❌ Rate limit test failed!")
        print(f"Error: {str(e)}")
        if "429" in str(e):
            print("ℹ️  This is a rate limit error. Your quota may be exceeded.")
        return False

# Test 5: Token Usage and Cost Estimation
def test_token_usage(client):
    """Test token usage reporting"""
    print("\n" + "="*70)
    print("  💰 Test 5: Token Usage & Cost Estimation")
    print("="*70)
    
    try:
        deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
        
        prompt = "Explain what TalentFlow AI does in one sentence."
        
        print("Testing token usage tracking...")
        
        response = client.chat.completions.create(
            model=deployment,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=100
        )
        
        usage = response.usage
        print(f"\n✅ Token usage tracked successfully!")
        print(f"  Prompt tokens: {usage.prompt_tokens}")
        print(f"  Completion tokens: {usage.completion_tokens}")
        print(f"  Total tokens: {usage.total_tokens}")
        
        # Rough cost estimation (GPT-4 pricing example)
        prompt_cost = (usage.prompt_tokens / 1000) * 0.03  # $0.03 per 1K prompt tokens
        completion_cost = (usage.completion_tokens / 1000) * 0.06  # $0.06 per 1K completion tokens
        total_cost = prompt_cost + completion_cost
        
        print(f"\n  💵 Estimated cost: ${total_cost:.6f}")
        print(f"     (Based on GPT-4 pricing: $0.03/1K prompt, $0.06/1K completion)")
        
        return True
        
    except Exception as e:
        print(f"\n❌ Token usage test failed!")
        print(f"Error: {str(e)}")
        return False

# Run all tests
def run_all_tests():
    """Run all Azure OpenAI configuration tests"""
    print("\n" + "="*70)
    print("  🧪 TalentFlow AI - Azure OpenAI Configuration Test Suite")
    print("="*70)
    print("\n")
    
    # Check environment variables
    if not check_env_vars():
        print("\n❌ Cannot proceed with tests. Please configure your .env file.")
        return
    
    # Initialize client
    print("\n📡 Initializing Azure OpenAI client...")
    client = init_client()
    
    if not client:
        print("\n❌ Cannot proceed with tests. Client initialization failed.")
        return
    
    print("✅ Client initialized successfully!")
    
    # Run tests
    results = {
        "Basic Connection": test_connection(client),
        "JD Generation": test_jd_generation(client),
        "AI Interview (JSON)": test_interview_json_mode(client),
        "Rate Limits": test_rate_limits(client),
        "Token Usage": test_token_usage(client)
    }
    
    # Summary
    print("\n" + "="*70)
    print("  📊 Test Summary")
    print("="*70)
    
    passed = sum(1 for v in results.values() if v)
    total = len(results)
    
    for test_name, result in results.items():
        status = "✅ PASS" if result else "❌ FAIL"
        print(f"  {status} - {test_name}")
    
    print(f"\n  Results: {passed}/{total} tests passed")
    
    if passed == total:
        print("\n  🎉 All tests passed! Your Azure OpenAI is configured correctly!")
        print("  🚀 TalentFlow AI is ready to use!")
    else:
        print("\n  ⚠️  Some tests failed. Please review the errors above.")
        print("  💡 Common issues:")
        print("     - Wrong deployment name")
        print("     - Invalid API key")
        print("     - Quota exceeded")
        print("     - Wrong API version")
    
    print("\n" + "="*70)
    
    return results

# Execute the test suite
if __name__ == "__main__":
    run_all_tests()



  🧪 TalentFlow AI - Azure OpenAI Configuration Test Suite


  🔍 Checking Environment Variables
  ✅ AZURE_OPENAI_API_KEY: DriisREt...W5PG
  ✅ AZURE_OPENAI_ENDPOINT: https://mazenetai27-9892-resource.openai.azure.com/
  ✅ AZURE_OPENAI_API_VERSION: 2024-12-01-preview
  ✅ AZURE_OPENAI_CHAT_DEPLOYMENT_NAME: gpt-4o-mini

✅ All environment variables are configured!

📡 Initializing Azure OpenAI client...
✅ Client initialized successfully!

  🌐 Test 1: Basic Connection
Deployment: gpt-4o-mini
Testing connection with a simple prompt...

❌ Connection failed!
Error: Error code: 401 - {'error': {'code': '401', 'message': 'Access denied due to invalid subscription key or wrong API endpoint. Make sure to provide a valid key for an active subscription and use a correct regional API endpoint for your resource.'}}

  📝 Test 2: Job Description Generation
Testing JD generation with sample data...

❌ JD Generation failed!
Error: Error code: 401 - {'error': {'code': '401', 'message': 'Access denied due to 

In [ ]:
# Run the Azure OpenAI configuration test suite
run_all_tests()


In [8]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:


endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
model_name = "gpt-4o-mini"
deployment = "chat-deployment"

subscription_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = "2024-12-01-preview"

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=subscription_key,
)

response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": "I am going to Paris, what should I see?",
        }
    ],
    max_tokens=4096,
    temperature=1.0,
    top_p=1.0,
    model=deployment
)

print(response.choices[0].message.content)

Paris is a beautiful city with a rich history, stunning architecture, and vibrant culture. Here are some must-see attractions and activities to consider during your visit:

1. **Eiffel Tower**: A trip to Paris isn't complete without visiting the iconic Eiffel Tower. You can go to the top for fantastic views of the city or enjoy a picnic in the nearby Champ de Mars.

2. **Louvre Museum**: As one of the largest and most famous art museums in the world, the Louvre is home to masterpieces like the Mona Lisa and the Venus de Milo.

3. **Notre-Dame Cathedral**: Although it suffered damage in the 2019 fire, Notre-Dame remains a symbol of Paris. You can admire its exterior and explore the Île de la Cité.

4. **Montmartre and the Basilica of Sacré-Cœur**: Wander through the charming streets of Montmartre, visit the Basilica of Sacré-Cœur for panoramic views of the city, and enjoy the artistic atmosphere.

5. **Champs-Élysées and Arc de Triomphe**: Stroll down the famous avenue, enjoy shopping o